# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.2, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [23]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [24]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster
Use findspark.init() to start a local cluster.  If you plan to use remote cluster, skip the findspark.init() and change the cluster_url according.

In [25]:
import findspark
findspark.init()

In [26]:
spark_url = 'local'

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [28]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

# Spark SQL Data Preparation

First, we read a csv file.  We can provide option such as delimiter and header.  We then rename the colume names to remove dot ('.') in the names.

In [29]:
path = 'bank-additional-full.csv'

In [30]:
df = spark.read.option("delimiter", ";").option("header", True).csv(path)

In [31]:
df.show(3)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
| 37| serv

In [32]:
df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [33]:
cols = [c.replace('.', '_') for c in df.columns]
cols

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp_var_rate',
 'cons_price_idx',
 'cons_conf_idx',
 'euribor3m',
 'nr_employed',
 'y']

In [34]:
df = df.toDF(*cols)

In [35]:
df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp_var_rate',
 'cons_price_idx',
 'cons_conf_idx',
 'euribor3m',
 'nr_employed',
 'y']

Check out data and schema

In [36]:
df.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
| 37| serv

In [37]:
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: string (nullable = true)
 |-- cons_price_idx: string (nullable = true)
 |-- cons_conf_idx: string (nullable = true)
 |-- euribor3m: string (nullable = true)
 |-- nr_employed: string (nullable = true)
 |-- y: string (nullable = true)



Spark SQL seems to not perform any guess on datatype.  To convert to proper data type, we cast each column to proper type using **'cast'** and replace back to the same column using **'withColumn'**.

In [38]:
df = df.withColumn('age', df.age.cast('int'))

In [39]:
cols = ['age', 'duration', 'campaign', 'pdays', 'previous', 'nr_employed']
for c in cols:
    df = df.withColumn(c, col(c).cast('int'))

In [40]:
cols = ['emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m']
for c in cols:
    df = df.withColumn(c, col(c).cast('double'))

Cast and also rename the column y to label

In [41]:
df = df.withColumn('label', df.y.cast('boolean'))

In [42]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: double (nullable = true)
 |-- cons_price_idx: double (nullable = true)
 |-- cons_conf_idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr_employed: integer (nullable = true)
 |-- y: string (nullable = true)
 |-- label: boolean (nullable = true)



# Basic Spark SQL Commands

We can select some columns using **'select'** and select some rows using **'filter'**.  Note that we can perform basic math to columns.

In [43]:
df.select(df['job'], df['education'], df['housing']).show(5)

+---------+-----------+-------+
|      job|  education|housing|
+---------+-----------+-------+
|housemaid|   basic.4y|     no|
| services|high.school|     no|
| services|high.school|    yes|
|   admin.|   basic.6y|     no|
| services|high.school|     no|
+---------+-----------+-------+
only showing top 5 rows



In [44]:
df.select(df['age'], df['duration'], df['pdays'], df['age']*2, df['duration']+df['pdays']).show(5)

+---+--------+-----+---------+------------------+
|age|duration|pdays|(age * 2)|(duration + pdays)|
+---+--------+-----+---------+------------------+
| 56|     261|  999|      112|              1260|
| 57|     149|  999|      114|              1148|
| 37|     226|  999|       74|              1225|
| 40|     151|  999|       80|              1150|
| 56|     307|  999|      112|              1306|
+---+--------+-----+---------+------------------+
only showing top 5 rows



In [45]:
df.filter(df.duration < 100).select('duration').show(3)

+--------+
|duration|
+--------+
|      50|
|      55|
|      38|
+--------+
only showing top 3 rows



In [46]:
df.filter(df['job'] == 'housemaid').select('job').show(5)

+---------+
|      job|
+---------+
|housemaid|
|housemaid|
|housemaid|
|housemaid|
|housemaid|
+---------+
only showing top 5 rows



In [47]:
df.filter((df['age'] > 60) & (df.age <= 65)).select('age', 'marital').show(5)

+---+--------+
|age| marital|
+---+--------+
| 61| married|
| 61| married|
| 61| married|
| 63|divorced|
| 62| married|
+---+--------+
only showing top 5 rows



In [48]:
df.filter("marital == 'married'").select('job', 'marital').show(5)

+---------+-------+
|      job|marital|
+---------+-------+
|housemaid|married|
| services|married|
| services|married|
|   admin.|married|
| services|married|
+---------+-------+
only showing top 5 rows



In [49]:
df.filter('age < 40 and duration > 200').select('age', 'duration', 'marital').show(5)

+---+--------+-------+
|age|duration|marital|
+---+--------+-------+
| 37|     226|married|
| 24|     380| single|
| 25|     222| single|
| 35|     312|married|
| 39|     233|married|
+---+--------+-------+
only showing top 5 rows



# Aggregate and Groupby Functions
We can use several built-in aggegrate functions.  We can also use groupby for group operations

In [50]:
from pyspark.sql.functions import avg, min, max, countDistinct

In [51]:
df.select(avg('age'), min('age'), max('duration')).show()

+-----------------+--------+-------------+
|         avg(age)|min(age)|max(duration)|
+-----------------+--------+-------------+
|40.02406040594348|      17|         4918|
+-----------------+--------+-------------+



Groupby function allows us to work data in groups.

In [52]:
df.groupby('marital').count().show()

+--------+-----+
| marital|count|
+--------+-----+
| unknown|   80|
|divorced| 4612|
| married|24928|
|  single|11568|
+--------+-----+



In [53]:
df.groupby('marital', 'education').agg({'age': 'min'}).show()

+--------+-------------------+--------+
| marital|          education|min(age)|
+--------+-------------------+--------+
|divorced|        high.school|      24|
|divorced|            unknown|      26|
| unknown|  university.degree|      25|
| unknown|            unknown|      31|
| married|professional.course|      22|
|  single|           basic.9y|      17|
| married|           basic.4y|      20|
| married|  university.degree|      23|
| unknown|           basic.9y|      30|
|divorced|           basic.4y|      25|
|divorced|           basic.9y|      24|
|  single|professional.course|      20|
| married|           basic.9y|      21|
|divorced|           basic.6y|      26|
| unknown|professional.course|      29|
|  single|           basic.6y|      18|
|  single|            unknown|      17|
| married|        high.school|      21|
| married|            unknown|      21|
| unknown|        high.school|      27|
+--------+-------------------+--------+
only showing top 20 rows



# User-Defined Function
We can create user-defined function using udf.

In [54]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [55]:
def agegroup_mapping(age): 
    if age < 25:
        return 'young'
    if age < 55:
        return 'adult'
    return 'senior'

to_agegroup = udf(agegroup_mapping, StringType())

In [56]:
df.select('age', to_agegroup('age')).show(5)

+---+---------------------+
|age|agegroup_mapping(age)|
+---+---------------------+
| 56|               senior|
| 57|               senior|
| 37|                adult|
| 40|                adult|
| 56|               senior|
+---+---------------------+
only showing top 5 rows



In [57]:
new_df = df.withColumn('agegroup', to_agegroup(df.age))
new_df.select(new_df['age'], new_df['agegroup']).show(10)

+---+--------+
|age|agegroup|
+---+--------+
| 56|  senior|
| 57|  senior|
| 37|   adult|
| 40|   adult|
| 56|  senior|
| 45|   adult|
| 59|  senior|
| 41|   adult|
| 24|   young|
| 25|   adult|
+---+--------+
only showing top 10 rows



In [58]:
spark.stop()